<a href="https://colab.research.google.com/github/soneekaacharyaa/recommend_news/blob/main/News_Recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import dask.dataframe as dd
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.figure_factory as ff
import plotly.graph_objects as go
import plotly.express as px
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder 
from sklearn.metrics import pairwise_distances
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
news_articles = pd.read_csv("/content/drive/MyDrive/newsdata.csv")
news_articles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209527 entries, 0 to 209526
Data columns (total 6 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   link               209527 non-null  object
 1   headline           209521 non-null  object
 2   category           209527 non-null  object
 3   short_description  189815 non-null  object
 4   authors            172109 non-null  object
 5   date               209527 non-null  object
dtypes: object(6)
memory usage: 9.6+ MB


In [ ]:
news_articles.head()

,link,headline,category,short_description,authors,date
0,https://www.huffpost.com/entry/covid-boosters-...,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...,"Carla K. Johnson, AP",2022-09-23
1,https://www.huffpost.com/entry/american-airlin...,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...,Mary Papenfuss,2022-09-23
2,https://www.huffpost.com/entry/funniest-tweets...,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",Elyse Wanshel,2022-09-23
3,https://www.huffpost.com/entry/funniest-parent...,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",Caroline Bologna,2022-09-23
4,https://www.huffpost.com/entry/amy-cooper-lose...,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...,Nina Golgowski,2022-09-22


In [ ]:
news_articles = news_articles[news_articles['date'] >= pd.Timestamp(2018,1,1)]

In [ ]:
news_articles


,link,headline,category,short_description,authors,date
0,https://www.huffpost.com/entry/covid-boosters-...,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...,"Carla K. Johnson, AP",2022-09-23
1,https://www.huffpost.com/entry/american-airlin...,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...,Mary Papenfuss,2022-09-23
2,https://www.huffpost.com/entry/funniest-tweets...,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",Elyse Wanshel,2022-09-23
3,https://www.huffpost.com/entry/funniest-parent...,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",Caroline Bologna,2022-09-23
4,https://www.huffpost.com/entry/amy-cooper-lose...,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...,Nina Golgowski,2022-09-22
...,...,...,...,...,...,...
209522,https://www.huffingtonpost.com/entry/rim-ceo-t...,RIM CEO Thorsten Heins' 'Significant' Plans Fo...,TECH,Verizon Wireless and AT&T are already promotin...,"Reuters, Reuters",2012-01-28
209523,https://www.huffingtonpost.com/entry/maria-sha...,Maria Sharapova Stunned By Victoria Azarenka I...,SPORTS,"Afterward, Azarenka, more effusive with the pr...",NaN,2012-01-28
209524,https://www.huffingtonpost.com/entry/super-bow...,"Giants Over Patriots, Jets Over Colts Among M...",SPORTS,"Leading up to Super Bowl XLVI, the most talked...",NaN,2012-01-28
209525,https://www.huffingtonpost.com/entry/aldon-smi...,Aldon Smith Arrested: 49ers Linebacker Busted ...,SPORTS,CORRECTION: An earlier version of this story i...,NaN,2012-01-28


In [ ]:
news_articles.shape

(17257, 6)

In [ ]:
news_articles = news_articles[news_articles['headline'].apply(lambda x: len(x.split())>5)]

In [ ]:
news_articles.sort_values('headline',inplace=True, ascending=False)
duplicated_articles_series = news_articles.duplicated('headline', keep = False)
news_articles = news_articles[~duplicated_articles_series]

In [ ]:
news_articles.isna().sum()

link                 0
headline             0
category             0
short_description    0
authors              0
date                 0
dtype: int64

In [ ]:
print(" articles : ", news_articles.shape[0])
print(" authors : ", news_articles["authors"].nunique())
print(" unqiue categories : ", news_articles["category"].nunique())

 articles :  17113
 authors :  2256
 unqiue categories :  36


In [ ]:
fig = go.Figure([go.Bar(x=news_articles["category"].value_counts().index, y=news_articles["category"].value_counts().values)])
fig['layout'].update(title={"text" : 'Distribution of articles category-wise','y':0.9,'x':0.5,'xanchor': 'center','yanchor': 'top'}, xaxis_title="Category name",yaxis_title="Number of articles")
fig.update_layout(width=800,height=700)
fig

In [ ]:
news_articles_per_month = news_articles.resample('m',on = 'date')['headline'].count()
news_articles_per_month

date
2018-01-31    2065
2018-02-28    1694
2018-03-31    1778
2018-04-30    1580
2018-05-31    1391
2018-06-30     136
2018-07-31     155
2018-08-31     123
2018-09-30     156
2018-10-31     182
2018-11-30     175
2018-12-31     180
2019-01-31     181
2019-02-28     168
2019-03-31     179
2019-04-30     147
2019-05-31     152
2019-06-30     153
2019-07-31     157
2019-08-31     161
2019-09-30     163
2019-10-31     183
2019-11-30     174
2019-12-31     181
2020-01-31     154
2020-02-29     132
2020-03-31     161
2020-04-30     170
2020-05-31     179
2020-06-30     176
2020-07-31     185
2020-08-31     180
2020-09-30     172
2020-10-31     175
2020-11-30     178
2020-12-31     181
2021-01-31     183
2021-02-28     140
2021-03-31     164
2021-04-30     176
2021-05-31     184
2021-06-30     176
2021-07-31     180
2021-08-31     175
2021-09-30     168
2021-10-31     174
2021-11-30     170
2021-12-31     170
2022-01-31     149
2022-02-28     152
2022-03-31     174
2022-04-30     162
2022-05

In [ ]:
fig = go.Figure([go.Bar(x=news_articles_per_month.index.strftime("%b"), y=news_articles_per_month)])
fig['layout'].update(title={"text" : 'Distribution of articles month-wise','y':0.9,'x':0.5,'xanchor': 'center','yanchor': 'top'}, xaxis_title="Month",yaxis_title="Number of articles")
fig.update_layout(width=500,height=500)
fig

In [ ]:
fig = ff.create_distplot([news_articles['headline'].str.len()], ["ht"],show_hist=False,show_rug=False)
fig['layout'].update(title={'text':'PDF','y':0.9,'x':0.5,'xanchor': 'center','yanchor': 'top'}, xaxis_title="Length of a headline",yaxis_title="probability")
fig.update_layout(showlegend = False,width=500,height=500)
fig

In [ ]:
news_articles.index = range(news_articles.shape[0])

In [ ]:
news_articles["day and month"] = news_articles["date"].dt.strftime("%a") + "_" + news_articles["date"].dt.strftime("%b")

In [ ]:
news_articles_temp = news_articles.copy()

In [ ]:
stop_words = set(stopwords.words('english'))

In [ ]:
for i in range(len(news_articles_temp["headline"])):
    string = ""
    for word in news_articles_temp["headline"][i].split():
        word = ("".join(e for e in word if e.isalnum()))
        word = word.lower()
        if not word in stop_words:
          string += word + " "  
    if(i%1000==0):
      print(i)           # To track number of records processed
    news_articles.at[i,"headline"] = string.strip()

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000


In [ ]:
lemmatizer = WordNetLemmatizer()

In [ ]:
lemmatizer = WordNetLemmatizer()
for i in range(len(news_articles_temp["headline"])):
    string = ""
    for w in word_tokenize(news_articles_temp["headline"][i]):
        string += lemmatizer.lemmatize(w,pos = "v") + " "
    news_articles.at[i, "headline"] = string.strip()
    if(i%1000==0):
        print(i)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000


In [ ]:
#headline_vectorizer = CountVectorizer()
#headline_features   = headline_vectorizer.fit_transform(news_articles_temp['headline'])

In [ ]:
#pd.set_option('display.max_colwidth', -1)

In [ ]:
#def bag_of_words_based_model(row_index, num_similar_items):
 #   couple_dist = pairwise_distances(headline_features,headline_features[row_index])
  #  indices = np.argsort(couple_dist.ravel())[0:num_similar_items]
   # df = pd.DataFrame({'publish_date': news_articles['date'][indices].values,
    #           'headline':news_articles['headline'][indices].values,
     #           'category': news_articles['category'][indices].values})
    #print("="*30,"Queried article details","="*30)
    #print('headline : ',news_articles['headline'][indices[0]])
    #print('category : ',news_articles['category'][indices[0]])
    #print("\n","="*25,"Recommended articles : ","="*23)
    #return df.iloc[1:,1]
    #return df

#bag_of_words_based_model(100, 5)

In [ ]:
tfidf_headline_vectorizer = TfidfVectorizer(min_df = 0)
tfidf_headline_features = tfidf_headline_vectorizer.fit_transform(news_articles_temp['headline'])

In [ ]:
def tfidf_based_model(row_index, num_similar_items):
  couple_dist = pairwise_distances(tfidf_headline_features,tfidf_headline_features[row_index])
  indices = np.argsort(couple_dist.ravel())[0:num_similar_items]
  df = pd.DataFrame({'publish_date': news_articles['date'][indices].values,
           'headline':news_articles['headline'][indices].values,
             'category': news_articles['category'][indices].values})
  print("="*30,"Queried article details","="*30)
  print('headline : ',news_articles['headline'][indices[0]])
  print("\n","="*25,"Recommended articles : ","="*23)
  return df.iloc[1:,]
tfidf_based_model(133, 5)

============================== Queried article details ==============================
headline :  Yovanovitch Responds To Trump Twitter Attacks In Real Time In Dramatic Impeachment Testimony

 ========================= Recommended articles :  =======================


,publish_date,headline,category
1,2019-11-15,Twitter Users Smack Down Trump 's Attack On Ma...,POLITICS
2,2019-11-16,Pelosi On Trump 's Yovanovitch Attack : He Kno...,POLITICS
3,2019-10-22,Trump Likens Impeachment Inquiry To ' A Lynchi...,POLITICS
4,2021-07-21,Dramatic Videos Show Deadly Flooding In China,ENVIRONMENT


In [ ]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle

In [ ]:
def make_lower_case(text):
    return text.lower()

In [ ]:
news_articles_temp['cleaned_headline'] = news_articles_temp['headline'].apply(func = make_lower_case)

In [ ]:
corpus = []
for words in news_articles_temp['cleaned_headline']:
    corpus.append(words.split())

corpus

[['’n',
  "sync's",
  'chris',
  'kirkpatrick',
  'says',
  'he',
  "'couldn't",
  'be',
  'in',
  'the',
  'same',
  "room'",
  'as',
  'this',
  'backstreet',
  'boy'],
 ['’19',
  'kids',
  'and',
  'counting’',
  'father',
  'running',
  'for',
  'arkansas',
  'senate'],
 ['‘world’s',
  'most',
  'dangerous',
  'bird’',
  'kills',
  '75-year-old',
  'owner',
  'in',
  'florida'],
 ['‘women',
  'for',
  'trump’',
  'cheer',
  'as',
  'president',
  'mocks',
  'christine',
  'blasey',
  'ford'],
 ['‘will',
  '&',
  'grace’',
  'creator',
  'to',
  'donate',
  'gay',
  'bunny',
  'book',
  'to',
  'every',
  'grade',
  'school',
  'in',
  'indiana'],
 ['‘where',
  'are',
  'the',
  'masks',
  'going?’',
  'trump',
  'questions',
  'use',
  'of',
  'supplies',
  'as',
  'coronavirus',
  'cases',
  'surge'],
 ['‘we’re',
  'back',
  'to',
  'square',
  'zero’',
  'on',
  'coronavirus,',
  'warns',
  'infectious',
  'disease',
  'expert'],
 ['‘trump',
  'grifts,',
  'you',
  'die’:',
  'ne

In [ ]:

model = Word2Vec(corpus,
  window=10,
  min_count=2,
  workers=4,
)
words = list(model.wv.vocab)
words

['chris',
 'says',
 'he',
 "'couldn't",
 'be',
 'in',
 'the',
 'same',
 "room'",
 'as',
 'this',
 'backstreet',
 'boy',
 'kids',
 'and',
 'father',
 'running',
 'for',
 'arkansas',
 'senate',
 'most',
 'dangerous',
 'kills',
 '75-year-old',
 'owner',
 'florida',
 '‘women',
 'cheer',
 'president',
 'mocks',
 'christine',
 'blasey',
 'ford',
 '‘will',
 '&',
 'grace’',
 'creator',
 'to',
 'donate',
 'gay',
 'bunny',
 'book',
 'every',
 'grade',
 'school',
 'indiana',
 'are',
 'masks',
 'trump',
 'questions',
 'use',
 'of',
 'supplies',
 'coronavirus',
 'cases',
 'surge',
 '‘we’re',
 'back',
 'square',
 'on',
 'coronavirus,',
 'warns',
 'infectious',
 'disease',
 'expert',
 '‘trump',
 'you',
 'new',
 'ad',
 'rips',
 'amid',
 'pandemic',
 'federal',
 'judge',
 'invalidates',
 'pipeline',
 '‘to',
 'say',
 'that',
 'she’s',
 'an',
 'abusive',
 'figure',
 'is',
 'at',
 'abc',
 'news,',
 '‘the',
 'blind',
 'auditions',
 'make',
 'history',
 'with',
 'first',
 'trans',
 'contestant',
 'smack',
 

In [ ]:
model.wv['concern']

array([-0.00466928, -0.07189084,  0.03637626, -0.02340018,  0.07219417,
        0.02079277, -0.02020335,  0.01860613,  0.05828512, -0.03083472,
       -0.1359998 , -0.07989759, -0.06071828,  0.02917524,  0.02350037,
        0.00968836,  0.04240676,  0.08342215,  0.03890251,  0.03394487,
        0.03103424, -0.04311841, -0.03182264,  0.03633091,  0.00712721,
        0.0227037 , -0.03011944,  0.00108333, -0.05017397,  0.00279127,
        0.03322462,  0.00047517,  0.03546879, -0.05143042,  0.03153398,
        0.02612709,  0.02703846,  0.02960054,  0.02807838,  0.05012338,
       -0.01453064,  0.05929445,  0.02458308, -0.0086583 ,  0.05841804,
       -0.01730756, -0.02897329,  0.05397774,  0.03745962, -0.00527847,
        0.092944  , -0.05991416,  0.02659041, -0.0556478 , -0.02066092,
        0.01424721,  0.03206687,  0.03501594,  0.01067399, -0.06247518,
        0.03505271,  0.01785115, -0.05961849,  0.00034455,  0.01091185,
       -0.03205964, -0.02853699, -0.03813407,  0.09234654,  0.01

In [ ]:
model.wv.n_similarity(['king','male'],['queen','female'])

0.9999181

In [ ]:
model.wv.similar_by_word('concern')

[('dwayne', 0.9977690577507019),
 ('defend', 0.997748851776123),
 ('hundreds', 0.9977378845214844),
 ('farm', 0.9976948499679565),
 ('reach', 0.9976938962936401),
 ('georgia', 0.9976771473884583),
 ('domestic', 0.9976704120635986),
 ('site', 0.9976654648780823),
 ('german', 0.9976565837860107),
 ('gaza', 0.9976538419723511)]

In [ ]:
model.save("model.h5")

In [ ]:
# Downloading the Google pretrained Word2Vec Model
#!wget -P /root/input/ -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

EMBEDDING_FILE = '/content/drive/MyDrive/GoogleNews-vectors-negative300.bin'
google_word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

google_model = Word2Vec(size = 300, window=5, min_count = 2, workers = -1)
google_model.build_vocab(corpus)
google_model.intersect_word2vec_format(EMBEDDING_FILE, lockf=1.0, binary=True)

google_model.train(corpus, total_examples=google_model.corpus_count, epochs = 5)

(0, 0)

In [ ]:
def vectors(x):
  global word_embeddings
  word_embeddings = []

    # Reading the each book description 
  for line in news_articles_temp['cleaned_headline']:
      avgword2vec = None
      count = 0
      for word in line.split():
          if word in model.wv.vocab:
                count += 1
                if avgword2vec is None:
                    avgword2vec = google_model[word]
                else:
                    avgword2vec = avgword2vec +google_model[word]
                
      if avgword2vec is not None:
            avgword2vec = avgword2vec / count
        
            word_embeddings.append(avgword2vec)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt

In [ ]:
# Pipeline
# 1. Get users current news content
# 2 . Create its embeddings using doc2vec model
# 3. Calculate similarity score with your database
##  a. Dataset with already calculated embeddings
##  b. Calculate using doc2vec methods
# 4. Show those recommendation

#References: 
#1. https://www.youtube.com/watch?v=GQbkthBPqV8
#2. https://medium.com/red-buffer/doc2vec-computing-similarity-between-the-documents-47daf6c828cd

In [ ]:
# Recommending the Top 5 similar news articles

def recommendations(headline):
    
    # Calling the function vectors

    vectors(news_articles_temp)
    cosine_similarities = cosine_similarity(word_embeddings, word_embeddings)
    news = news_articles_temp[['headline']]
    indices = pd.Series(news_articles_temp.index, index = news_articles_temp['headline']).drop_duplicates()
         
    idx = indices[headline]
    sim_scores = list(enumerate(cosine_similarities[idx]))
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)
    sim_scores = sim_scores[1:6]
    book_indices = [i[0] for i in sim_scores]
    recommend = news.iloc[book_indices]
    for index, row in recommend.iterrows():
      print(row)
      # Why are plotting title only ? just print the recommended values !!

In [ ]:
recommendations('World Cup Captains Want To Wear Rainbow Armbands In Qatar')

<ipython-input-41-8c82bd819cca>:13: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).

<ipython-input-41-8c82bd819cca>:15: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).



headline    Iceland Holds Off Argentina In 1-1 Draw At Fir...
Name: 10343, dtype: object
headline    Sikh Man Rocks Rainbow Turban For Pride And Th...
Name: 4463, dtype: object
headline    Ramadan's End Gives Muslim Players Reprieve As...
Name: 5718, dtype: object
headline    Shop The Trend: How To Wear The Lemon Print Yo...
Name: 4487, dtype: object
headline    Stars Wear All Black At Golden Globes In Power...
Name: 4223, dtype: object
